In [1]:
# attempting SVC (as ANN results were pathetic, accuracy around 0.68)
# tried various ML classifiers but nothing really works well
# not sure if the EEG signals are really good at detecting confusion
# ok - all this is without including the subjectID or videoID 
# pretty sure this will change with these two, esp with subjectID 
# trying this in next file 

In [38]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn import svm
from sklearn.model_selection import GridSearchCV, cross_val_predict
from sklearn.metrics import confusion_matrix, classification_report, f1_score

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC

In [3]:
eeg_df = pd.read_csv('EEG_data.csv')

In [4]:
eeg_df.describe()

,SubjectID,VideoID,Attention,Mediation,Raw,Delta,Theta,Alpha1,Alpha2,Beta1,Beta2,Gamma1,Gamma2,predefinedlabel,user-definedlabeln
count,12811.000000,12811.000000,12811.000000,12811.000000,12811.000000,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,1.281100e+04,12811.000000,12811.000000
mean,4.487394,4.390602,41.313871,47.182656,65.570760,6.057853e+05,1.680526e+05,4.138435e+04,3.318339e+04,2.431837e+04,3.814433e+04,2.959255e+04,1.441597e+04,0.479978,0.512606
std,2.865373,2.913232,23.152953,22.655976,597.921035,6.376236e+05,2.441346e+05,7.243082e+04,5.831410e+04,3.837968e+04,7.906606e+04,7.982637e+04,3.603523e+04,0.499618,0.499861
min,0.000000,0.000000,0.000000,0.000000,-2048.000000,4.480000e+02,1.700000e+01,2.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,2.000000e+00,0.000000,0.000000
25%,2.000000,2.000000,27.000000,37.000000,-14.000000,9.806400e+04,2.691750e+04,6.838000e+03,6.852000e+03,6.140000e+03,7.358500e+03,4.058000e+03,2.167500e+03,0.000000,0.000000
50%,4.000000,4.000000,43.000000,51.000000,35.000000,3.954870e+05,8.133100e+04,1.750000e+04,1.495900e+04,1.281800e+04,1.581000e+04,9.763000e+03,5.116000e+03,0.000000,1.000000
75%,7.000000,7.000000,57.000000,63.000000,90.000000,9.166230e+05,2.052760e+05,4.477950e+04,3.455050e+04,2.740600e+04,3.549400e+04,2.488800e+04,1.266950e+04,1.000000,1.000000
max,9.000000,9.000000,100.000000,100.000000,2047.000000,3.964663e+06,3.007802e+06,1.369955e+06,1.016913e+06,1.067778e+06,1.645369e+06,1.972506e+06,1.348117e+06,1.000000,1.000000


In [5]:
demo_df = pd.read_csv('demographic_info.csv')

In [6]:
# merging data frames on subject ID - but different nomenclature, so first rename in second df

In [7]:
demo_df.rename(columns={'subject ID':'SubjectID'}, inplace=True)

In [8]:
df = demo_df.merge(eeg_df, on='SubjectID')

In [9]:
# cleaning up column names which are in different formats
# fuckall extraspace and extra n
df.rename(columns={' age':'Age', ' ethnicity':'Ethnicity', ' gender':'Gender', 'user-definedlabeln':'Label'}, inplace=True)

In [10]:
# dropping subjectid, videoid and one of the labels predefinedlabel as they are not useful
df = df.drop(['SubjectID', 'VideoID', 'predefinedlabel'], axis=1)

In [11]:
# label is in floats - can change it to ints - looks better
df['Label'] = df['Label'].astype(np.int)

In [12]:
# encoding features

In [13]:
# replacing gender with 1 for M & 0 for F
df['Gender'] = df['Gender'].apply(lambda x:1 if x == 'M' else 0)

In [14]:
# getting dummies for ethnicity
ethnic_dummies = pd.get_dummies(df['Ethnicity']) #, drop_first=True)
df = pd.concat([df, ethnic_dummies], axis=1)
df = df.drop('Ethnicity', axis=1)

In [15]:
# scaling and splitting data
X = df.drop('Label', axis=1).copy()
y = df['Label'].copy()

In [16]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [18]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8967, 16), (3844, 16), (8967,), (3844,))

In [20]:
svc = svm.SVC()
svc.fit(X_train, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

In [21]:
svc.score(X_train, y_train)

0.6774841083974573

In [22]:
y_pred_svc = svc.predict(X_test)

In [26]:
print(classification_report(y_test, y_pred_svc))

              precision    recall  f1-score   support

           0       0.66      0.63      0.64      1902
           1       0.65      0.68      0.67      1942

    accuracy                           0.65      3844
   macro avg       0.66      0.65      0.65      3844
weighted avg       0.65      0.65      0.65      3844



In [ ]:
# bah this is as bad as the ANN
# perhaps i should try the other logreg/random forest etc etc algorithms 

In [28]:
# Log Reg

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [30]:
logreg.score(X_train, y_train)

0.5967436154789785

In [31]:
y_pred_logreg = logreg.predict(X_test)
print(classification_report(y_test, y_pred_logreg))

              precision    recall  f1-score   support

           0       0.58      0.59      0.58      1902
           1       0.59      0.59      0.59      1942

    accuracy                           0.59      3844
   macro avg       0.59      0.59      0.59      3844
weighted avg       0.59      0.59      0.59      3844



In [32]:
# Random Forest Classifier

rf = RandomForestClassifier(n_estimators=100)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [34]:
rf.score(X_train, y_train)

1.0

In [35]:
y_pred_rf = rf.predict(X_test)
print(classification_report(y_test, y_pred_rf))

              precision    recall  f1-score   support

           0       0.70      0.66      0.68      1902
           1       0.69      0.73      0.71      1942

    accuracy                           0.70      3844
   macro avg       0.70      0.69      0.69      3844
weighted avg       0.70      0.70      0.69      3844



In [36]:
# still meh for a RF

In [39]:
# Stochastic Gradient Descent Classifier

sgd = SGDClassifier(max_iter=5, tol=None)
sgd.fit(X_train, y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=5,
              n_iter_no_change=5, n_jobs=None, penalty='l2', power_t=0.5,
              random_state=None, shuffle=True, tol=None,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [40]:
sgd.score(X_train, y_train)

0.5129920820787331

In [41]:
y_pred_sgd = sgd.predict(X_test)
print(classification_report(y_test, y_pred_sgd))

              precision    recall  f1-score   support

           0       0.52      0.38      0.44      1902
           1       0.52      0.66      0.58      1942

    accuracy                           0.52      3844
   macro avg       0.52      0.52      0.51      3844
weighted avg       0.52      0.52      0.51      3844



In [43]:
# SVC - Linear version used in class - attn

In [44]:
linear_svc = LinearSVC()
linear_svc.fit(X_train, y_train)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [45]:
linear_svc.score(X_train, y_train)

0.5961860153897625

In [46]:
y_pred_linsvc = linear_svc.predict(X_test)
print(classification_report(y_test, y_pred_linsvc))

              precision    recall  f1-score   support

           0       0.58      0.59      0.58      1902
           1       0.59      0.59      0.59      1942

    accuracy                           0.59      3844
   macro avg       0.59      0.59      0.59      3844
weighted avg       0.59      0.59      0.59      3844



In [47]:
# decision tree classifier

dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [48]:
dt.score(X_train, y_train)

1.0

In [49]:
y_pred_dt = dt.predict(X_test)
print(classification_report(y_test, y_pred_dt))

              precision    recall  f1-score   support

           0       0.61      0.58      0.59      1902
           1       0.61      0.64      0.62      1942

    accuracy                           0.61      3844
   macro avg       0.61      0.61      0.61      3844
weighted avg       0.61      0.61      0.61      3844



In [ ]:
# all these algorithms give a sad sad performance....
# or well, EEG is crap... 